### Import libraries

In [7]:
import pandas as pd
import statsmodels.formula.api as smf
import joblib

### Load the dataset

In [3]:
data = pd.read_csv("./datasets/doggy-boot-harness.csv")

data.head()

,boot_size,harness_size,sex,age_years
0,39,58,male,12.0
1,38,58,male,9.6
2,37,52,female,8.6
3,39,58,male,10.2
4,38,57,male,7.8


### Create and train a model

In [6]:
model = smf.ols(formula='boot_size ~ harness_size', data=data).fit()

print("Model trained!")

Model trained!


### Save and load a model

In [8]:
model_filename = './avalanche_dog_boot_model.pkl'
joblib.dump(model, model_filename)

print("Model saved!")

model_loaded = joblib.load(model_filename)

print("We have loaded a model with the following parameters:")
print(model_loaded.params)

Model saved!
We have loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64


### Put it together

In [10]:
# Let's write a function that loads and uses our model

def load_model_and_predict(harness_size):
    '''
    This function loads a pretrained model. It uses the model 
    with the customer's dog's harness size to predict the size of 
    boots that will fit that dog.

    harness_size: The dog harness size, in cm
    '''

    # Load the model from and print basic information about it
    loaded_model = joblib.load('./avalanche_dog_boot_model.pkl')

    print("We've loaded a model with the following parameters:")
    print(loaded_model.params)

    # Prepare data for the model
    inputs = {'harness_size': harness_size}

    # Use the model to make a prediction
    predicted_boot_size = loaded_model.predict(inputs)

    return predicted_boot_size

# Practice using our model
predicted_boot_size = load_model_and_predict(45)

print("Predicted dog boot size:", predicted_boot_size[0])

We've loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64
Predicted dog boot size: 32.085753565904774


In [12]:
def check_size_of_boots(selected_harness_size, selected_boot_size):
    '''
    Calculates whether the customer has chosen a pair of doggy boots that 
    are a sensible size. This works by estimating the dog's actual boot 
    size from their harness size.

    This returns a message for the customer that should be shown before
    they complete their payment 

    selected_harness_size: The size of the harness the customer wants to buy
    selected_boot_size: The size of the doggy boots the customer wants to buy
    '''

    # Estimate the customer's dog's boot size
    estimated_boot_size = load_model_and_predict(selected_harness_size)

    # Round to the nearest whole number because we don't sell partial sizes
    estimated_boot_size = int(round(estimated_boot_size[0]))

    # Check if the boot size selected is appropriate
    if selected_boot_size == estimated_boot_size:
        # The selected boots are probably OK
        return f"Great choice! We think these boots will fit your avalanche dog well."

    if selected_boot_size < estimated_boot_size:
        # Selected boots might be too small 
        return "The boots you have selected might be TOO SMALL for a dog as "\
               f"big as yours. We recommend a doggy boots size of {estimated_boot_size}."

    if selected_boot_size > estimated_boot_size:
        # Selected boots might be too big 
        return "The boots you have selected might be TOO BIG for a dog as "\
               f"small as yours. We recommend a doggy boots size of {estimated_boot_size}."
    

# Practice using our new warning system
check_size_of_boots(selected_harness_size=55, selected_boot_size=39)

We've loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64


'The boots you have selected might be TOO BIG for a dog as small as yours. We recommend a doggy boots size of 38.'